In [ ]:
from bc_algos.dataset.robomimic import RobomimicDataset
from bc_algos.utils.constants import Modality, GoalMode
import bc_algos.utils.obs_utils as ObsUtils
import matplotlib.pyplot as plt
import numpy as np

def display(img):
    if not isinstance(img, list):
        img = [img]
    _, axs = plt.subplots(1, len(img))
    for i in range(len(img)):
        axs[i].imshow(img[i].astype(int))
    plt.show()

### Normalization Utils Unit Tests

In [ ]:
T = 10
state_shape = [2, 2]

traj0_dict = {
    "state": 2*np.random.randn(T, *state_shape)-1
}
traj1_dict = {
    "state": 2*np.random.randn(T, *state_shape)-1
}

In [ ]:
traj0_stats = ObsUtils.compute_traj_stats(traj0_dict)
traj1_stats = ObsUtils.compute_traj_stats(traj1_dict)
merged_stats = ObsUtils.aggregate_traj_stats(traj0_stats, traj1_stats)
traj0_norm_stats = ObsUtils.compute_normalization_stats(traj0_stats)
merged_norm_stats = ObsUtils.compute_normalization_stats(merged_stats)

traj0_state = traj0_dict["state"]
assert np.allclose(traj0_norm_stats["state"]["mean"], traj0_state.mean(axis=0))
assert np.allclose(traj0_norm_stats["state"]["stdv"], np.std(traj0_state, axis=0))
merged_state = np.concatenate((traj0_dict["state"], traj1_dict["state"]), axis=0)
assert np.allclose(merged_norm_stats["state"]["mean"], merged_state.mean(axis=0))
assert np.allclose(merged_norm_stats["state"]["stdv"], np.std(merged_state, axis=0))

### Dataset Init

In [ ]:
obs_key_to_modality = {"robot0_eef_pos": Modality.LOW_DIM, "robot0_eef_quat": Modality.LOW_DIM, "agentview_image": Modality.RGB}
obs_group_to_key = {"obs": ["robot0_eef_pos", "robot0_eef_quat", "agentview_image"], 
                     "goal": ["agentview_image"]}
dataset_keys = ["actions"]
dataset_path = "../datasets/test/square_ph.hdf5"
demos = ["demo_0", "demo_1"]
frame_stack = 1
seq_length = 2

In [ ]:
dataset = RobomimicDataset(
    path=dataset_path,
    obs_key_to_modality=obs_key_to_modality,
    obs_group_to_key=obs_group_to_key, 
    dataset_keys=dataset_keys, 
    frame_stack=frame_stack,
    seq_length=seq_length,
    pad_frame_stack=False,
    pad_seq_length=False,
    get_pad_mask=False,
    demos=demos,
)

### Padding Unit Tests

In [ ]:
dataset_pad = RobomimicDataset(
    path=dataset_path,
    obs_key_to_modality=obs_key_to_modality,
    obs_group_to_key=obs_group_to_key, 
    dataset_keys=dataset_keys, 
    frame_stack=frame_stack,
    seq_length=seq_length,
    demos=demos,
)

for i in range(frame_stack):
    pad_mask = dataset_pad[i]["pad_mask"]
    gt_mask = np.array([0] * (frame_stack-i) + [1] * (seq_length+i))
    assert np.all(np.equal(pad_mask, gt_mask))
for i in range(1, seq_length):
    pad_mask = dataset_pad[-i]["pad_mask"]
    gt_mask = np.array([1] * (frame_stack+i) + [0] * (seq_length-i))
    assert np.all(np.equal(pad_mask, gt_mask))

### Sequence Fetching Unit Tests

In [ ]:
for i in range(len(dataset)):
    frame = dataset[i]
    T = frame["actions"].shape[0]
    assert T == frame_stack + seq_length
    for j in range(T-1):
        for obs_key in frame["obs"]:
            assert np.any(np.not_equal(frame["obs"][obs_key][j], frame["obs"][obs_key][j+1]))
        assert np.any(np.not_equal(frame["actions"][j], frame["actions"][j+1]))
    
for i in range(len(dataset)-1):
    if dataset.index_to_demo_id[i] == dataset.index_to_demo_id[i+1]:
        frame_a = dataset[i]
        frame_b = dataset[i+1]
        T = frame_a["actions"].shape[0]
        for j in range(T-1):
            for obs_key in frame["obs"]:
                assert np.all(np.equal(frame_a["obs"][obs_key][j+1], frame_b["obs"][obs_key][j]))
            assert np.all(np.equal(frame_a["actions"][j+1], frame_b["actions"][j]))

### Goal-Conditioning Unit Tests

#### Last Goal

In [ ]:
dataset.goal_mode = GoalMode.LAST
dataset.cache_index()

for i in range(len(dataset)):
    frame = dataset[i]
    for obs_key in frame["obs"]:
        T_obs = frame["obs"][obs_key].shape[0]
        assert T_obs == frame_stack + seq_length
    for obs_key in frame["goal"]:
        T_goal = frame["goal"][obs_key].shape[0]
        assert T_goal == 1

for i in range(len(dataset)-1):
    if dataset.index_to_demo_id[i] == dataset.index_to_demo_id[i+1]:
        frame_a = dataset[i]
        frame_b = dataset[i+1]
        for obs_key in frame_a["goal"]:
            goal_a = frame_a["goal"][obs_key]
            goal_b = frame_b["goal"][obs_key]
            assert np.all(np.equal(goal_a, goal_b))

display([goal_a[0], goal_b[0]])

#### Dense Subgoals

In [ ]:
dataset.goal_mode = GoalMode.SUBGOAL
dataset.num_subgoal = None
dataset.cache_index()

for i in range(len(dataset)):
    frame = dataset[i]
    for obs_key in frame["obs"]:
        T_obs = frame["obs"][obs_key].shape[0]
        assert T_obs == frame_stack + seq_length
    for obs_key in frame["goal"]:
        T_goal = frame["goal"][obs_key].shape[0]
        assert T_goal == seq_length

for i in range(len(dataset)-1):
    if dataset.index_to_demo_id[i] == dataset.index_to_demo_id[i+1]:
        frame_a = dataset[i]
        frame_b = dataset[i+1]
        for obs_key in frame_a["goal"]:
            goal_a = frame_a["goal"][obs_key]
            goal_b = frame_b["goal"][obs_key]
            assert np.any(np.not_equal(goal_a, goal_b))
            assert np.all(np.equal(goal_a[1:], goal_b[:-1]))

display([goal_a[0], goal_b[0]])

#### Sparse Subgoals

In [ ]:
num_subgoal = 3

In [ ]:
dataset.goal_mode = GoalMode.SUBGOAL
dataset.num_subgoal = num_subgoal
dataset.cache_index()

frame_a = dataset[0]
frame_b = dataset[1]
frame_c = dataset[-1]

for obs_key in frame_a["obs"]:
    T_obs = frame_a["obs"][obs_key].shape[0]
    assert T_obs == frame_stack + seq_length
for obs_key in frame_a["goal"]:
    T_goal = frame_a["goal"][obs_key].shape[0]
    assert T_goal == seq_length

for obs_key in frame_a["goal"]:
    goal_a = frame_a["goal"][obs_key]
    goal_b = frame_b["goal"][obs_key]
    goal_c = frame_c["goal"][obs_key]
    assert np.all(np.equal(goal_a, goal_b))
    assert np.any(np.not_equal(goal_b, goal_c))

display([goal_a[0], goal_b[0], goal_c[0]])

#### Full Subgoals

In [ ]:
dataset.goal_mode = GoalMode.FULL
dataset.num_subgoal = num_subgoal
dataset.cache_index()

for i in range(len(dataset)):
    frame = dataset[i]
    for obs_key in frame["obs"]:
        T_obs = frame["obs"][obs_key].shape[0]
        assert T_obs == frame_stack + seq_length
    for obs_key in frame["goal"]:
        T_goal = frame["goal"][obs_key].shape[0]
        assert T_goal == num_subgoal

for i in range(len(dataset)-1):
    if dataset.index_to_demo_id[i] == dataset.index_to_demo_id[i+1]:
        frame_a = dataset[i]
        frame_b = dataset[i+1]
        for obs_key in frame_a["goal"]:
            goal_a = frame_a["goal"][obs_key]
            goal_b = frame_b["goal"][obs_key]
            assert np.all(np.equal(goal_a, goal_b))

display(list(goal_a))